## Preambule

In [1]:
# General packages
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import xarray as xr
import pandas as pd
import json
import plotly

# Plotting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors

## Parameters

In [2]:
rules = ['PCC', 'ECPC', 'AP']
rulecolors = ['goldenrod', 'tomato', 'forestgreen']

## Paths

In [3]:
path_main = Path("K:/Code/effort-sharing/")
path_data = Path("K:/Data/Data_effortsharing/DataUpdate_ongoing/")
path_figs = path_main / "Figures" / "Paper_FairShares"

## Read data files

In [4]:
xr_dataread = xr.open_dataset(path_data / "startyear_2021/xr_dataread.nc")
xr_traj_2030 = xr.open_dataset(path_data / "startyear_2021/Aggregated_files/xr_alloc_2030_GHG_incl.nc")
xr_traj_2040 = xr.open_dataset(path_data / "startyear_2021/Aggregated_files/xr_alloc_2040_GHG_incl.nc")
xr_total = xr.open_dataset(path_data / "xr_policyscen.nc")

all_regions_iso = np.load(path_data / "all_regions.npy")
all_regions_names = np.load(path_data / "all_regions_names.npy")
all_countries_iso = np.load(path_data / "all_countries.npy", allow_pickle=True)
all_countries_names = np.load(path_data / "all_countries_names.npy", allow_pickle=True)

## Get Cost-optimal output

In [5]:
df_ar6 = pd.read_csv("X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_ISO3_v1.1.csv")
df_ar6 = df_ar6[df_ar6.Variable.isin(['Emissions|Kyoto Gases', 'Policy Cost|Consumption Loss'])]
df_ar6 = df_ar6.reset_index(drop=True)

df_ar6_meta = pd.read_excel("X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx", sheet_name='meta_Ch3vetted_withclimate')
mods = np.array(df_ar6_meta.Model)
scens = np.array(df_ar6_meta.Scenario)
modscens_meta = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_ar6_meta['ModelScenario'] = modscens_meta
df_ar6_meta = df_ar6_meta[['ModelScenario', 'Category', 'Policy_category']]
df_ar6_meta = df_ar6_meta[df_ar6_meta.Category.isin(['C1', 'C2'])]
df_ar6_meta = df_ar6_meta.reset_index(drop=True)
ms_meta_refined = np.array(df_ar6_meta.ModelScenario)

mods = np.array(df_ar6.Model)
scens = np.array(df_ar6.Scenario)
modscens = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_ar6['ModelScenario'] = modscens
df_ar6 = df_ar6.drop(['Model', 'Scenario', 'Unit'], axis=1)
df_ar6 = df_ar6[df_ar6.ModelScenario.isin(np.array(ms_meta_refined))]

dummy = df_ar6.melt(id_vars=["Variable", "Region", "ModelScenario"], var_name="Time", value_name="Value")
dummy['Time'] = np.array(dummy['Time'].astype(int))
dummy = dummy.set_index(["Variable", "Region", "ModelScenario", "Time"])
xr_scen_r = xr.Dataset.from_dataframe(dummy)
xr_scen_r = xr_scen_r.reindex(Time = np.arange(1850, 2101))
xr_scen_r = xr_scen_r.interpolate_na(dim="Time", method="linear")

In [6]:
cost_opt = xr_scen_r.sel(Variable='Emissions|Kyoto Gases', Time=2040)/xr_scen_r.sel(Variable='Emissions|Kyoto Gases', Time=2040)*xr_scen_r.sel(Variable='Emissions|Kyoto Gases', Time=2040)

## Compute fractions

In [7]:
xr_larger_than_costopt = xr_traj_2040.sel(Temperature=1.5, Convergence_year=2050, Risk=0.5, NegEmis=0.5, NonCO2red=0.5, Timing='Immediate').median(dim=['Scenario']) > cost_opt.mean(dim='ModelScenario').Value

World

In [8]:
df_ar6 = pd.read_csv("X:/user/dekkerm/Data/IPCC/AR6_Scenarios_Database_World_v1.1.csv")
df_ar6 = df_ar6[df_ar6.Variable.isin(['Emissions|Kyoto Gases', 'Policy Cost|Consumption Loss'])]
df_ar6 = df_ar6.reset_index(drop=True)

df_ar6_meta = pd.read_excel("X:/user/dekkerm/Data/IPCC/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx", sheet_name='meta_Ch3vetted_withclimate')
mods = np.array(df_ar6_meta.Model)
scens = np.array(df_ar6_meta.Scenario)
modscens_meta = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_ar6_meta['ModelScenario'] = modscens_meta
df_ar6_meta = df_ar6_meta[['ModelScenario', 'Category', 'Policy_category']]
df_ar6_meta = df_ar6_meta[df_ar6_meta.Category.isin(['C1', 'C2'])]
df_ar6_meta = df_ar6_meta.reset_index(drop=True)
ms_meta_refined = np.array(df_ar6_meta.ModelScenario)

mods = np.array(df_ar6.Model)
scens = np.array(df_ar6.Scenario)
modscens = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_ar6['ModelScenario'] = modscens
df_ar6['Region'] = 'WORLD'
df_ar6 = df_ar6.drop(['Model', 'Scenario', 'Unit'], axis=1)
df_ar6 = df_ar6[df_ar6.ModelScenario.isin(np.array(ms_meta_refined))]

dummy = df_ar6.melt(id_vars=["Variable", "Region", "ModelScenario"], var_name="Time", value_name="Value")
dummy['Time'] = np.array(dummy['Time'].astype(int))
dummy = dummy.set_index(["Variable", "Region", "ModelScenario", "Time"])
xr_scen = xr.Dataset.from_dataframe(dummy)
xr_scen = xr_scen.reindex(Time = np.arange(1850, 2101))
xr_scen = xr_scen.interpolate_na(dim="Time", method="linear")

Regional

In [9]:
df_ar6 = pd.read_csv("X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_ISO3_v1.1.csv")
df_ar6 = df_ar6[df_ar6.Variable.isin(['Emissions|Kyoto Gases', 'Policy Cost|Consumption Loss'])]
df_ar6 = df_ar6.reset_index(drop=True)

df_ar6_meta = pd.read_excel("X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx", sheet_name='meta_Ch3vetted_withclimate')
mods = np.array(df_ar6_meta.Model)
scens = np.array(df_ar6_meta.Scenario)
modscens_meta = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_ar6_meta['ModelScenario'] = modscens_meta
df_ar6_meta = df_ar6_meta[['ModelScenario', 'Category', 'Policy_category']]
df_ar6_meta = df_ar6_meta[df_ar6_meta.Category.isin(['C1', 'C2'])]
df_ar6_meta = df_ar6_meta.reset_index(drop=True)
ms_meta_refined = np.array(df_ar6_meta.ModelScenario)

mods = np.array(df_ar6.Model)
scens = np.array(df_ar6.Scenario)
modscens = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_ar6['ModelScenario'] = modscens
df_ar6 = df_ar6.drop(['Model', 'Scenario', 'Unit'], axis=1)
df_ar6 = df_ar6[df_ar6.ModelScenario.isin(np.array(ms_meta_refined))]

dummy = df_ar6.melt(id_vars=["Variable", "Region", "ModelScenario"], var_name="Time", value_name="Value")
dummy['Time'] = np.array(dummy['Time'].astype(int))
dummy = dummy.set_index(["Variable", "Region", "ModelScenario", "Time"])
xr_scen_r = xr.Dataset.from_dataframe(dummy)
xr_scen_r = xr_scen_r.reindex(Time = np.arange(1850, 2101))
xr_scen_r = xr_scen_r.interpolate_na(dim="Time", method="linear")

Concatenate

In [10]:
xr_scen_all = xr.merge([xr_scen, xr_scen_r])

## Data for plots

In [11]:
settings_default = {'Temperature': 1.6,
                   'Risk': 0.5,
                   'NegEmis': 0.5,
                   'NonCO2red': 0.5,
                   'Timing': "Immediate",
                   'Convergence_year': 2050,
                   'Scenario': 'SSP2',
                   'Discount_factor': 0,
                   'Historical_startyear': 1950,}
                   #'Capability_threshold': "Th",
                   #'RCI_weight': "Half"}
settings_ranges = {'Temperature': [1.6, 2.0],
                   'Risk': [0.5, 0.33],
                   'Convergence_year': [2050, 2080],
                    'NonCO2red':[0.33, 0.5, 0.67],
                   'Timing': ["Immediate"],
                   }
                   #'Capability_threshold': "Th",
                   #'RCI_weight': "Half"}

In [12]:
ndcs = xr_dataread.GHG_ndc.mean(dim=['Conditionality', 'Ambition'])
data = xr_traj_2030.sel(**settings_ranges)[['ECPC', 'AP', 'PCC']]
data_max = xr.concat([data.ECPC, data.AP.expand_dims({'Convergence_year': [2050, 2080]}), data.PCC], dim='variable2').max(dim=list(settings_default.keys())+['variable2'])
data = xr_traj_2030.sel(**settings_default)[['ECPC', 'AP', 'PCC']]
data_def = xr.concat([data.ECPC, data.AP, data.PCC], dim='variable2').max(dim='variable2')

In [13]:
print("Preferences on default settings")
print('-----')
data = xr_traj_2030.sel(**settings_default)[['ECPC', 'AP', 'PCC']]
data_def = xr.concat([data.ECPC, data.AP, data.PCC], dim='variable2').max(dim='variable2')
argmax_default = xr.concat([data.ECPC, data.AP, data.PCC], dim='variable2').fillna(-9e99).argmax(dim='variable2')

data = xr_traj_2030.sel(**settings_ranges)[['ECPC', 'AP', 'PCC']]
data_max = xr.concat([data.ECPC, data.AP.expand_dims({'Convergence_year': [2050, 2080]}), data.PCC], dim='variable2').max(dim=list(settings_default.keys())+['variable2'])
argmax_maximum = xr.concat([data.ECPC, data.AP.expand_dims({'Convergence_year': [2050, 2080]}), data.PCC], dim='variable2').fillna(-9e99).argmax(dim=list(settings_default.keys())+['variable2'])
x = xr.concat([data.ECPC, data.AP.expand_dims({'Convergence_year': [2050, 2080]}), data.PCC], dim='variable2').fillna(-9e99)
argmax_max = []
maxs = x.max(dim=list(settings_default.keys())+['variable2'])
for cty in x.Region:
    mx = float(maxs.sel(Region=cty).values)
    wh = np.where(x.sel(Region=cty).values == mx)
    wh = np.array(wh)[:, 0]
    argmax_max.append(wh[0])
argmax_max = np.array(argmax_max)

df_c = []
df_r_d = []
df_r_m = []
for cty_i, cty in enumerate(all_countries_iso):
    df_c.append(all_countries_names[cty_i])
    df_r_d.append(['ECPC', 'AP', 'PCC'][argmax_default.sel(Region=cty).values])
    df_r_m.append(['ECPC', 'AP', 'PCC'][argmax_max[cty_i]])
df_dict = {'Region': df_c, 'Default': df_r_d, 'Maximum': df_r_m}
df = pd.DataFrame(df_dict)
df.to_csv(path_data / "preferences_default_max.csv")

Preferences on default settings
-----


In [14]:
xr_policyscen = xr.open_dataset(path_data / "xr_policyscen.nc")
curpols = xr_policyscen.CurPol.sel(Time=2030).mean(dim='Model')
nzs = xr_policyscen.NetZero.sel(Time=2030).mean(dim='Model')

In [15]:
path_ctygroups = "X:/user/dekkerm/Data/" + "UNFCCC_Parties_Groups_noeu.xlsx"
df = pd.read_excel(path_ctygroups, sheet_name = "Country groups")
countries_iso = np.array(df["Country ISO Code"])
countries_name = np.array(df["Name"])
countries_iso = np.array(df["Country ISO Code"])
group_eu = countries_iso[np.array(df["EU"]) == 1]

In [16]:
costopts = xr_scen_all.sel(Time=2030, Variable='Emissions|Kyoto Gases').mean(dim='ModelScenario')

## Gaps

In [17]:
nec = xr_dataread.GHG_globe.sel(Time=2030, Temperature=1.6, Risk=0.5, NonCO2red=0.5, NegEmis=0.5, Timing='Immediate').values
gap_default = float(data_def.sel(Region=countries_iso).sum() - nec)
gap_default_perc = float((data_def.sel(Region=countries_iso).sum() - nec)/nec)

gap_max = float(data_max.sel(Region=countries_iso).sum() - nec)
gap_max_perc = float((data_max.sel(Region=countries_iso).sum() - nec)/nec)

In [18]:
xr_alloc_2030 = xr.open_dataset(path_data / "startyear_2021/Aggregated_files/xr_alloc_2030_GHG_incl.nc")

## New plot

In [19]:
fig = make_subplots(rows=3, cols=2,
                    specs=[[{'type': 'choropleth'}, {'type': 'choropleth'}],
                           [{'type': 'choropleth'}, {'type': 'choropleth'}],
                           [{}, {}]],
                    horizontal_spacing = 0., vertical_spacing=0.02)
for i in range(4):
    dath = [(ndcs-data_def)/data_def,
            (ndcs-data_max)/data_max,
            (costopts.Value)/data_def-1,
            (costopts.Value)/data_max-1][i]
    dat = np.array(dath)
    # if i>2:
    #     dat[dat < 0] = 0
    #scale = ['RdBu_r', 'RdBu_r', 'PuOr_r', 'PuOr_r'][i]
    scale = ['RdYlBu_r', 'RdYlBu_r', 'Earth_r', 'Earth_r'][i]
    dc = [dict(orientation='h', x=0.5, y=0.675, xanchor='center', yanchor='bottom', title='NDC deviation from emission allocation in 2030', tickvals=[-1, -0.5, 0, 0.5, 1],
                        ticktext=['100%<br>NDC sufficient <', '50%', '0%', '50%', '100%<br>> NDC insufficient'],
                        tickfont=dict(size=14),
                        len=0.4, lenmode='fraction', thickness=20, thicknessmode='pixels', titlefont=dict(size=14)),
          dict(orientation='h', x=0.5, y=0.675, xanchor='center', yanchor='bottom', title='NDC deviation from emission allocation in 2030', tickvals=[-1, -0.5, 0, 0.5, 1],
                        ticktext=['100%<br>NDC sufficient <', '50%', '0%', '50%', '100%<br>> NDC insufficient'],
                        tickfont=dict(size=14),
                        len=0.4, lenmode='fraction', thickness=20, thicknessmode='pixels', titlefont=dict(size=14)),
        dict(orientation='h', x=0.5, y=0.34, xanchor='center', yanchor='bottom', title='Fraction of mitigation in 2030 non-domestic', tickvals=[-1, -0.5, 0, 0.5, 1],
                        ticktext=['100%<br>More allocated than cost-optimal', '50%', '0%', '50%', '100%<br>Less allocated than cost-optimal'],
                        tickfont=dict(size=14),
                        len=0.4, lenmode='fraction', thickness=20, thicknessmode='pixels', titlefont=dict(size=14)),
        dict(orientation='h', x=0.5, y=0.34, xanchor='center', yanchor='bottom', title='Fraction of mitigation in 2030 non-domestic', tickvals=[-1, -0.5, 0, 0.5, 1],
                        ticktext=['100%<br>More allocated than cost-optimal', '50%', '0%', '50%', '100%<br>Less allocated than cost-optimal'],
                        tickfont=dict(size=14),
                        len=0.4, lenmode='fraction', thickness=20, thicknessmode='pixels', titlefont=dict(size=14))][i]
    
    # Grey out nans
    if i < 2:
        wh = np.where(np.isnan(dat))[0]
        fig.add_trace(go.Choropleth(
                locations=list(np.array(dath.Region)[wh])+['GRL'],
                z = [-0.5]*(len(dat[wh])+1),
                locationmode = 'ISO-3',
                colorscale ='Greys',
                zmax = 1,
                zmin = -1,
                hovertemplate  = '%{text}',
                name="",
                marker_line_color='black', 
                marker_line_width=0,
                showscale=False,
            ), [1, 1, 2, 2][i], [1, 2, 1, 2][i])
    else:
        fig.add_trace(go.Choropleth(
                locations=list(np.array(data_def.Region))+['GRL'],
                z = [-0.5]*(len(np.array(data_def.Region))+1),
                locationmode = 'ISO-3',
                colorscale ='Greys',
                zmax = 1,
                zmin = -1,
                hovertemplate  = '%{text}',
                name="",
                marker_line_color='black', 
                marker_line_width=0,
                showscale=False,
            ), [1, 1, 2, 2][i], [1, 2, 1, 2][i])
    
    fig.add_trace(go.Choropleth(
            locations=np.array((dath).Region),
            z = dat,
            locationmode = 'ISO-3',
            colorscale = scale,
            zmax = 1,
            zmin = -1,
            text = [str(r)+": "+str(np.round(100*dat[r_i], 1))+" %" for r_i, r in enumerate(np.array((dath).Region))],
            hovertemplate  = '%{text}',
            name="",
            colorbar=dc,
            marker_line_color='black', 
            marker_line_width=0.1,
            showscale=True,
        ), [1, 1, 2, 2][i], [1, 2, 1, 2][i])
    
    # EU
    fig.add_trace(go.Choropleth(
            locations=group_eu,
            z = [dat[np.where(np.array((dath).Region) == 'EU')[0][0]]]*len(group_eu),
            locationmode = 'ISO-3',
            colorscale =scale,
            zmax = 1,
            zmin = -1,
            #text = [str(r)+": "+str(np.round(100*dat[r_i], 1))+" %" for r_i, r in enumerate(group_eu)],
            hovertemplate  = '%{text}',
            name="",
            marker_line_color='black', 
            marker_line_width=0.1,
            showscale=False,
        ), [1, 1, 2, 2][i], [1, 2, 1, 2][i])
    
    # Emphasize >0
    wh = np.where(np.array(dath) > 0)[0]
    fig.add_trace(go.Choropleth(
            locations=np.array((dath).Region)[wh],
            z = dat[wh],
            locationmode = 'ISO-3',
            colorscale =scale,
            zmax = 1,
            zmin = -1,
            text = [str(r)+": "+str(np.round(100*dat[wh][r_i], 1))+" %" for r_i, r in enumerate(np.array((dath).Region)[wh])],
            hovertemplate  = '%{text}',
            name="",
            marker_line_color='black', 
            marker_line_width=2,
            showscale=False,
        ), [1, 1, 2, 2][i], [1, 2, 1, 2][i])
    
    if 'EU' in np.array((dath).Region)[wh]:
        fig.add_trace(go.Choropleth(
                locations=group_eu,
                z = [dat[np.where(np.array((dath).Region) == 'EU')[0][0]]]*len(group_eu),
                locationmode = 'ISO-3',
                colorscale =scale,
                zmax = 1,
                zmin = -1,
                #text = [str(r)+": "+str(np.round(100*dat[r_i], 1))+" %" for r_i, r in enumerate(group_eu)],
                hovertemplate  = '%{text}',
                name="",
                marker_line_color='black', 
                marker_line_width=2,
                showscale=False,
        ), [1, 1, 2, 2][i], [1, 2, 1, 2][i])

fs=20
fig['layout'].update(
    annotations=[dict(x=0.03, y=1.02, text='<b>(a) Comparison with NDCs</b>', xref='paper', yref='paper', showarrow=False, font=dict(color='black', size=fs)),
                 dict(x=0.53, y=1.02, text='<b>(b) Comparison with NDCs', xref='paper', yref='paper', xanchor='left', showarrow=False, font=dict(color='black', size=fs)),
                 dict(x=0.03, y=0.68, text='<b>(c) Comparison with cost-optimal pathways', xref='paper', yref='paper', xanchor='left', showarrow=False, font=dict(color='black', size=fs)),
                 dict(x=0.53, y=0.68, text='<b>(d) Comparison with cost-optimal pathways', xref='paper', yref='paper', xanchor='left', showarrow=False, font=dict(color='black', size=fs)),
                 dict(x=0.03, y=0.325, text='<b>(e) Domestic mitigation and international mitigation financing', xref='paper', yref='paper', xanchor='left', showarrow=False, font=dict(color='black', size=fs)),
                # dict(x=0.53, y=0.325, text='<b>(f) Current debt or inequality', xref='paper', yref='paper', xanchor='left', showarrow=False, font=dict(color='black', size=fs)),
                 dict(x=0.5, y=-0.05, text='Emissions reductions by 2030 w.r.t. 2015', xref='paper', yref='paper', xanchor='center', showarrow=False, font=dict(color='black', size=18)),

                 dict(x=0.03, y=1.07, text='<i>Default allocation settings: gap is {} Gt CO<sub>2</sub>e (+{}%) in 2030'.format(np.round(gap_default/1e3, 1), int(np.round(gap_default_perc*100, 0))), xref='paper', yref='paper', xanchor='left', showarrow=False, font=dict(color='black', size=fs)),
                 dict(x=0.53, y=1.07, text='<i>Maximum allocation settings: gap is {} Gt CO<sub>2</sub>e (+{}%) in 2030'.format(np.round(gap_max/1e3, 1), int(np.round(gap_max_perc*100, 0))), xref='paper', yref='paper', xanchor='left', showarrow=False, font=dict(color='black', size=fs)),

                 dict(x=0.03, y=0.984, text='(default)', xref='paper', yref='paper', xanchor='left', showarrow=False, font=dict(color='black', size=fs/1.5)),
                 dict(x=0.03, y=0.47, text='(default)', xref='paper', yref='paper', xanchor='left', showarrow=False, font=dict(color='black', size=fs/1.5)),
                 dict(x=0.53, y=0.984, text='(maximum)', xref='paper', yref='paper', xanchor='left', showarrow=False, font=dict(color='black', size=fs/1.5)),
                 dict(x=0.53, y=0.47, text='(maximum)', xref='paper', yref='paper', xanchor='left', showarrow=False, font=dict(color='black', size=fs/1.5)),

                #  dict(x=0.0, y=0.12, text='<b>United States</b> has 53% more<br>cost-optimal emissions (in 2030)<br>than would be fair to emit<br>on default settings', xref='paper', yref='paper', xanchor='left', showarrow=False, font=dict(color='black', size=14)),
                #  dict(x=0.99, y=0.04, text='When looking at the highest potentially<br>fair allocation for <b>Australia</b>,<br>this country is allocated equal emissions<br>as would be cost-optimal to emit here', xref='paper', yref='paper', xanchor='right', showarrow=False, font=dict(color='black', size=14)),
                #  dict(x=0.83, y=0.25, text='<b>Russia</b> is always allocated less<br>than is cost-optimal to emit there', xref='paper', yref='paper', xanchor='right', showarrow=False, font=dict(color='black', size=14)),
                #  dict(x=0.95, y=0.59, text='All reddish/bold countries here have<br>insufficient NDCs, even when looking<br>at the highest potentially fair allocation', xref='paper', yref='paper', xanchor='right', showarrow=False, font=dict(color='black', size=14)),
                #  dict(x=0.0, y=0.59, text='Most of the Global North together<br>with countries like Australia and Argentina<br>have insufficient NDCs when compared<br>to default fair share allocations', xref='paper', yref='paper', xanchor='left', showarrow=False, font=dict(color='black', size=14)),
                 ])

fig.update_geos(visible=False,
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                projection_type='natural earth',
                resolution=110,
                showcoastlines=False,)
fig.update_traces(colorbar=dict(
    title=dict(
        side="bottom"
    )
))
fig.update_geos(landcolor="grey")
args = dict(xref="paper", yref="paper", type='line')
# fig.add_shape(x0=0.08, y0=0.20, x1=0.12, y1=0.35, **args, line=dict(color='black', width=2))
# fig.add_shape(x0=0.75, y0=0.29, x1=0.85, y1=0.40, **args, line=dict(color='black', width=2))
# fig.add_shape(x0=0.90, y0=0.12, x1=0.93, y1=0.18, **args, line=dict(color='black', width=2))

# fig.add_shape(x0=0.08, y0=0.63, x1=0.12, y1=0.85, **args, line=dict(color='black', width=2))
# fig.add_shape(x0=0.87, y0=0.62, x1=0.85, y1=0.90, **args, line=dict(color='black', width=2))


fig.add_shape(x0=0., y0=1.03, x1=0.495, y1=1.03, **args, line=dict(color='black', width=2))
fig.add_shape(x0=0., y0=1.01, x1=0., y1=1.03, **args, line=dict(color='black', width=2))
fig.add_shape(x0=0.495, y0=1.01, x1=0.495, y1=1.03, **args, line=dict(color='black', width=2))

fig.add_shape(x0=0.505, y0=1.03, x1=1, y1=1.03, **args, line=dict(color='black', width=2))
fig.add_shape(x0=0.505, y0=1.01, x1=0.505, y1=1.03, **args, line=dict(color='black', width=2))
fig.add_shape(x0=1, y0=1.01, x1=1, y1=1.03, **args, line=dict(color='black', width=2))

fig.update_layout(height=1500, width=1700)

a = 0 
b = 0
# Bar plot
for col_i, col in enumerate([1, 2]):
    categories = [['JPN', 'RUS', 'SAU', 'EU', 'BRA', 'USA'],
                   ['PAK', 'IND', 'CHN', 'MEX', 'IDN', 'ZAF']][col_i]
    names = categories# [all_countries_names[np.where(all_countries_iso == i)[0][0]] for i in categories]
    emis_2015 = xr_dataread.GHG_hist.sel(Time=2015, Region=names)
    sub_values_1 = np.array(1-xr_alloc_2030.sel(Region=names, Temperature=1.6, Risk=0.5, Timing='Immediate', NonCO2red=0.5, NegEmis=0.5, Scenario='SSP2', Convergence_year=2050, Discount_factor=0, Historical_startyear=1990).PCC/emis_2015)    # Equality
    sub_values_2 = np.array(1-xr_alloc_2030.sel(Region=names, Temperature=1.6, Risk=0.5, Timing='Immediate', NonCO2red=0.5, NegEmis=0.5, Scenario='SSP2', Convergence_year=2050, Discount_factor=0, Historical_startyear=1990).AP/emis_2015)    # Second set of values
    sub_values_3 = np.array(1-xr_alloc_2030.sel(Region=names, Temperature=1.6, Risk=0.5, Timing='Immediate', NonCO2red=0.5, NegEmis=0.5, Scenario='SSP2', Convergence_year=2050, Discount_factor=0, Historical_startyear=1990).ECPC/emis_2015)    # Third set of values

    # Create a horizontal bar chart
    fig.add_trace(go.Bar(
        x=sub_values_1,
        y=names,
        name='Per capita convergence<br>(Equality)',
        orientation='h',
        marker_color='steelblue',
        showlegend=[True, False][col_i],
        opacity=0.3
    ), 3, col)

    # Add the second set of bars
    fig.add_trace(go.Bar(
        x=sub_values_2,
        y=names,
        name='Ability to pay<br>(Capability)',
        orientation='h',
        marker_color='goldenrod',
        showlegend=[True, False][col_i],
        opacity=0.3
    ), 3, col)

    # Add the third set of bars
    fig.add_trace(go.Bar(
        x=sub_values_3,
        y=names,
        name='Equal cumulative per capita<br>(Responsibility)',
        orientation='h',
        marker_color='purple',
        showlegend=[True, False][col_i],
        opacity=0.3
    ), 3, col)

    for i, category in enumerate(names):
        # Calculate the x-position for the vertical line (sum of all subcategory values for this category)
        total_min = float(1-(xr_dataread.GHG_ndc / emis_2015).sel(Region=category).min())
        total_max = float(1-(xr_dataread.GHG_ndc / emis_2015).sel(Region=category).max())
        costopt = float(1-costopts.sel(Region=category).Value/emis_2015.sel(Region=category))
        min_fairshare = np.min([sub_values_1[i], sub_values_2[i], sub_values_3[i]])
        wd = 0.4
        # Add a vertical line at the total position
        fig.add_shape(
            type="line",
        col=col, row=3,
            x0=total_min, x1=total_min,
            y0=-wd + i, y1=wd + i,
            name='NDC',
            showlegend=[[True, False, False, False, False, False, False][i], False][col_i],
            line=dict(color="forestgreen", width=5)
        )
        fig.add_shape(
            type="line",
        col=col, row=3,
            x0=total_max, x1=total_max,
            y0=-wd + i, y1=wd + i,
            line=dict(color="forestgreen", width=5)
        )
        fig.add_shape(
        col=col, row=3,
            type="line",
            x0=total_min, x1=total_max,
            y0=i, y1=i,
            line=dict(color="forestgreen", width=5)
        )

        fig.add_shape(
            type="line",
        col=col, row=3,
            x0=costopt, x1=costopt,
            y0=-wd + i, y1=wd + i,
            name='Cost optimal',
            showlegend=[[True, False, False, False, False, False, False][i], False][col_i],
            line=dict(color="chocolate", width=5)
        )

        fig.add_shape(
            type="line",
            col=col, row=3,
            x0=min_fairshare, x1=min_fairshare,
            y0=-wd + i, y1=wd + i,
            name='Least-stringent fair target',
            showlegend=[[True, False, False, False, False, False, False][i], False][col_i],
            line=dict(color="black", width=5)
        )

        # Increase ambition
        if min_fairshare > total_min:
            if costopt < min_fairshare and costopt > total_min:
                fig.add_shape(
                    row = 3, col=col,
                    type='rect',  # Draw a rectangle
                    x0=total_min, x1=costopt,  # The range of the box (between min and max values)
                    y0=i - 0.4, y1=i + 0.4,  # Position the box on the y-axis (with some vertical padding)
                    line=dict(color='black', width=0),  # Border color and width of the box
                    name='Increase domestic ambition',
                    showlegend=[[True, False, False, False, False, False, False][i], False][col_i],
                    opacity = 0.5,
                    fillcolor='forestgreen',  # Transparent fill color (adjust alpha for transparency)
                )
            elif costopt > min_fairshare:
                fig.add_shape(
                    row = 3, col=col,
                    type='rect',  # Draw a rectangle
                    x0=total_min, x1=min_fairshare,  # The range of the box (between min and max values)
                    y0=i - 0.4, y1=i + 0.4,  # Position the box on the y-axis (with some vertical padding)
                    line=dict(color='black', width=0),  # Border color and width of the box
                    opacity = 0.5,
                    fillcolor='forestgreen',  # Transparent fill color (adjust alpha for transparency)
                )
        # Pay abroad
        if min_fairshare > total_min:
            if costopt < total_min:
                fig.add_shape(
                    row = 3, col=col,
                    type='rect',  # Draw a rectangle
                    x0=total_min, x1=min_fairshare,  # The range of the box (between min and max values)
                    y0=i - 0.4, y1=i + 0.4,  # Position the box on the y-axis (with some vertical padding)
                    line=dict(color='black', width=0),  # Border color and width of the box
                    name='Pay for mitigation abroad',
                    showlegend=[True, False, False, False, False, False, False, False, False, False][a],
                    opacity = 0.5,
                    fillcolor='tomato',  # Transparent fill color (adjust alpha for transparency)
                )
                a+=1
            elif costopt < min_fairshare:
                fig.add_shape(
                    row = 3, col=col,
                    type='rect',  # Draw a rectangle
                    x0=costopt, x1=min_fairshare,  # The range of the box (between min and max values)
                    y0=i - 0.4, y1=i + 0.4,  # Position the box on the y-axis (with some vertical padding)
                    line=dict(color='black', width=0),  # Border color and width of the box
                    opacity = 0.5,
                    fillcolor='tomato',  # Transparent fill color (adjust alpha for transparency)
                )
        # Receive from abroad
        if min_fairshare < costopt:
            fig.add_shape(
                row = 3, col=col,
                type='rect',  # Draw a rectangle
                x0=total_min, x1=costopt,  # The range of the box (between min and max values)
                y0=i - 0.4, y1=i + 0.4,  # Position the box on the y-axis (with some vertical padding)
                line=dict(color='black', width=0),  # Border color and width of the box
                name='Receive mitigation payment from abroad',
                showlegend=[True, False, False, False, False, False, False, False, False, False][b],
                opacity = 0.3,
                fillcolor='grey',  # Transparent fill color (adjust alpha for transparency)
            )
            b+=1

            # elif costopt > min_fairshare:
            #     fig.add_shape(
            #         row = 3, col=col,
            #         type='rect',  # Draw a rectangle
            #         x0=total_min, x1=min_fairshare,  # The range of the box (between min and max values)
            #         y0=i - 0.4, y1=i + 0.4,  # Position the box on the y-axis (with some vertical padding)
            #         line=dict(color='black', width=1),  # Border color and width of the box
            #         opacity = 0.3,
            #         fillcolor='forestgreen',  # Transparent fill color (adjust alpha for transparency)
            #     )
    fig.update_yaxes(row=3, col=col,tickfont=dict(size=16))
    fig.update_layout(
        plot_bgcolor='white',  # Background of the plot area
        paper_bgcolor='white',  # Background of the entire figure
        barmode='group',  # Ensures the bars are grouped
        # yaxis=dict(
        #     categoryorder='array',  # Fix the order of the categories
        #     categoryarray=categories  # Set the desired order of the categories explicitly
        # ),,
       
        
        legend=dict(
            x=0.6,  # Horizontal position (right side)
            y=0.3,  # Vertical position (bottom)
            xanchor='center',  # Anchor horizontally at the center of the legend box
            yanchor='top',     # Anchor vertically at the top of the legend box
        )
    )
    fig.update_xaxes(row=3, col=col, range=[(-1, 3), (-1.5, 1.5)][col_i], tickfont=dict(size=16),
                     tickvals=[[-1, -0.5, 0, 0.5, 1, 1.5, 2, 2.5], [-2, -1.5, -1, -0.5, 0, 0.5, 1, 1.5]][col_i],
                     ticktext=[['-100%', '-50%', 'No<br>reduction', '50%', '100%', '150%', '200%', '250%'], ['-200%', '-150%', '-100%', '-50%', 'No<br>reduction', '50%', '100%', '150%']][col_i])
    fig.add_shape(
        col=col, row=3,
        type="line",
        x0=0, x1=0,
        y0=-0.5, y1=len(names)-0.5,
        line=dict(color="black", width=3)
    )
#fig.write_image("K:/Code/effort-sharing/Figures/Paper_FairShares/V2/Figure_4.png", scale=5)
fig.show()